In [1]:
import random
import numbers
from PIL import Image, ImageMath
import os
import os.path
import numpy as np
import struct
import math

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

import time
import itertools

import pcl
import timeit
import png

import multiprocessing

In [2]:
def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:, 1] - extents[:, 0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize / 2
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)


def plot_pc(pc_np, z_cutoff=1000, birds_view=False, color='height', size=0.3, ax=None, cmap=cm.jet, is_equal_axes=True):
    # remove large z points
    valid_index = pc_np[:, 0] < z_cutoff
    pc_np = pc_np[valid_index, :]

    if ax is None:
        fig = plt.figure(figsize=(9, 9))
        ax = Axes3D(fig)
    if type(color)==str and color == 'height':
        c = pc_np[:, 2]
        ax.scatter(pc_np[:, 0], pc_np[:, 1], pc_np[:, 2], s=size, c=c, cmap=cmap, edgecolors='none')
    elif type(color)==str and color == 'reflectance':
        assert False
    elif type(color) == np.ndarray:
        ax.scatter(pc_np[:, 0], pc_np[:, 1], pc_np[:, 2], s=size, c=color, cmap=cmap, edgecolors='none')
    else:
        ax.scatter(pc_np[:, 0], pc_np[:, 1], pc_np[:, 2], s=size, c=color, edgecolors='none')

    if is_equal_axes:
        axisEqual3D(ax)
    if True == birds_view:
        ax.view_init(elev=0, azim=-90)
    else:
        ax.view_init(elev=-45, azim=-90)
    # ax.invert_yaxis()

    return ax


def read_txt_as_list(txt_file):
    with open(txt_file, 'r') as f:
        subset = [x.rstrip() for x in f.readlines()]
    return subset


def read_camera_intrinsics(txt_file):
    return np.loadtxt(txt_file)

def img2pc(depth, camera_matrix):
    """Transform a depth image into a point cloud with one point for each
    pixel in the image, using the camera transform for a camera
    centred at cx, cy with field of view fx, fy.

    depth is a 2-D ndarray with shape (rows, cols) containing
    depths from 1 to 254 inclusive. The result is a 3-D array with
    shape (rows, cols, 3). Pixels with invalid depth in the input have
    NaN for the z-coordinate in the result.

    """
    fx = camera_matrix[0, 0]
    fy = camera_matrix[1, 1]
    cx = camera_matrix[0, 2]
    cy = camera_matrix[1, 2]
    
    rows, cols = depth.shape
    c, r = np.meshgrid(np.arange(cols), np.arange(rows), sparse=True)
    valid = (depth > 1)
    z = np.where(valid, depth / 1000.0, 0)
    x = np.where(valid, z * (c - cx) / fx, 0)
    y = np.where(valid, z * (r - cy) / fy, 0)
    
    pc_HxWx3 = np.dstack((x, y, z))
    pc = pc_HxWx3.reshape(rows*cols, 3)
    valid_idx = pc[:, 2] > 1e-3
    pc = pc[valid_idx, :]
    
    return pc


# get surface normal
def Surface_normals(cloud):
    ne = cloud.make_NormalEstimation()
    tree = cloud.make_kdtree()
    ne.set_SearchMethod(tree)
#     ne.set_RadiusSearch(2)
    ne.set_KSearch(9)
    cloud_normals = ne.compute()
    return cloud_normals

In [ ]:
# single thread
dataset_root = '/data/datasets/3DMatch_train'
output_root = '/data/datasets/3DMatch_npy'
downsample_size = 16384

train_list = read_txt_as_list(os.path.join(dataset_root, 'training_list.txt'))

for folder in train_list:
    # load camera intrinsics
    camera_matrix = read_camera_intrinsics(os.path.join(dataset_root, folder, 'camera-intrinsics.txt'))
    
    # find subfolder
    things_in_folder = os.listdir(os.path.join(dataset_root, folder));
    for subfolder in things_in_folder:
        if not os.path.isdir(os.path.join(dataset_root, folder, subfolder)):
            continue
        # print info
        print(os.path.join(dataset_root, folder, subfolder))
        
        # find files
        things_in_subfolder = os.listdir(os.path.join(dataset_root, folder, subfolder))
        for file in things_in_subfolder:
            if 'depth.png' in file:
                png_file = os.path.join(dataset_root, folder, subfolder, file)
                npy_folder = os.path.join(output_root, folder, subfolder)
                if not os.path.isdir(npy_folder):
                    os.makedirs(npy_folder)
                npy_file = os.path.join(npy_folder, file[0:-4]+'.npy')
                
                png_reader = png.Reader(png_file)
                png_data = png_reader.read()
                png_np = np.vstack(list(map(np.uint16, png_data[2])))
                
                pc_np = img2pc(png_np, camera_matrix)
                # downsample
                if pc_np.shape[0] > downsample_size:
                    choice_idx = np.random.choice(pc_np.shape[0], downsample_size, replace=False)
                    pc_np = pc_np[choice_idx, :]
                    
                # compute surface normal
                cloud = pcl.PointCloud(pc_np.astype(np.float32))    
                sn = Surface_normals(cloud)
                sn_np = np.asarray(sn.to_array(), dtype=np.float32)  # Nx4, nx,ny,nz,curvature

                output_np = np.concatenate((pc_np.astype(np.float32), sn_np), axis=1)  # Nx7
                np.save(npy_file, output_np)
                
                # visualize
                # plot_pc(pc_np)
                # plt.show()
                # break


In [9]:
# multithread
total_thread_num = 4
def save_npy(thread_idx):
    dataset_root = '/data/datasets/3DMatch_train'
    output_root = '/data/datasets/3DMatch_npy'
    downsample_size = 16384

    train_list = read_txt_as_list(os.path.join(dataset_root, 'testing_list.txt'))
    
    chunk_size = math.floor(len(train_list) / total_thread_num)
    if thread_idx != total_thread_num-1:
        begin_idx = thread_idx * chunk_size
        end_idx = (thread_idx+1) * chunk_size
    else:
        begin_idx = thread_idx * chunk_size
        end_idx = len(train_list)

    for folder in train_list[begin_idx:end_idx]:
        # load camera intrinsics
        camera_matrix = read_camera_intrinsics(os.path.join(dataset_root, folder, 'camera-intrinsics.txt'))

        # find subfolder
        things_in_folder = os.listdir(os.path.join(dataset_root, folder));
        for subfolder in things_in_folder:
            if not os.path.isdir(os.path.join(dataset_root, folder, subfolder)):
                continue
            # print info
            print(os.path.join(dataset_root, folder, subfolder))

            # find files
            things_in_subfolder = os.listdir(os.path.join(dataset_root, folder, subfolder))
            for file in things_in_subfolder:
                if 'depth.png' in file:
                    png_file = os.path.join(dataset_root, folder, subfolder, file)
                    npy_folder = os.path.join(output_root, folder, subfolder)
                    if not os.path.isdir(npy_folder):
                        os.makedirs(npy_folder)
                    npy_file = os.path.join(npy_folder, file[0:-4]+'.npy')

                    png_reader = png.Reader(png_file)
                    png_data = png_reader.read()
                    png_np = np.vstack(list(map(np.uint16, png_data[2])))

                    pc_np = img2pc(png_np, camera_matrix)
                    # downsample
                    if pc_np.shape[0] > downsample_size:
                        choice_idx = np.random.choice(pc_np.shape[0], downsample_size, replace=False)
                        pc_np = pc_np[choice_idx, :]

                    # compute surface normal
                    cloud = pcl.PointCloud(pc_np.astype(np.float32))    
                    sn = Surface_normals(cloud)
                    sn_np = np.asarray(sn.to_array(), dtype=np.float32)  # Nx4, nx,ny,nz,curvature

                    output_np = np.concatenate((pc_np.astype(np.float32), sn_np), axis=1)  # Nx7
                    np.save(npy_file, output_np)

                    # visualize
                    # plot_pc(pc_np)
                    # plt.show()
                    # break

p_list = []
for i in range(total_thread_num):
    p_list.append(0)
for i in range(total_thread_num):
    p_list[i] = multiprocessing.Process(target=save_npy, args=(i, ))
    p_list[i].start()
    
for i in range(total_thread_num):
    p_list[i].join()

/data/datasets/3DMatch_train/7-scenes-redkitchen/seq-06
/data/datasets/3DMatch_train/sun3d-hotel_umd-maryland_hotel1/seq-01
/data/datasets/3DMatch_train/sun3d-home_md-home_md_scan9_2012_sep_30/seq-01
/data/datasets/3DMatch_train/sun3d-mit_76_studyroom-76-1studyroom2/seq-01
/data/datasets/3DMatch_train/7-scenes-redkitchen/seq-04
/data/datasets/3DMatch_train/7-scenes-redkitchen/seq-03
/data/datasets/3DMatch_train/7-scenes-redkitchen/seq-11
/data/datasets/3DMatch_train/sun3d-mit_lab_hj-lab_hj_tea_nov_2_2012_scan1_erika/seq-01
/data/datasets/3DMatch_train/7-scenes-redkitchen/seq-05
/data/datasets/3DMatch_train/7-scenes-redkitchen/seq-13
/data/datasets/3DMatch_train/sun3d-hotel_umd-maryland_hotel3/seq-01
/data/datasets/3DMatch_train/7-scenes-redkitchen/seq-07
/data/datasets/3DMatch_train/7-scenes-redkitchen/seq-02
/data/datasets/3DMatch_train/7-scenes-redkitchen/seq-14
/data/datasets/3DMatch_train/7-scenes-redkitchen/seq-12
/data/datasets/3DMatch_train/7-scenes-redkitchen/seq-01
/data/datas